# PCD - Laboratoire 7 - Recommendation musicale basée sur le contenu

In [1]:
import numpy as np
import pandas as pd

from itertools import combinations
from sklearn.preprocessing import normalize
from scipy.stats import spearmanr

from pyAudioAnalysis import ShortTermFeatures
from pyAudioAnalysis import MidTermFeatures

# 1. Extraction des données

Les chansons ont été téléchargées dans un dossier `data/`.

Extraction des attributs globaux à moyen terme (moyenne et écart-type) de chaque chanson:

In [2]:
songs = MidTermFeatures.directory_feature_extraction("data", 1, 1, 0.05, 0.05) # Valeurs issues de l'exemple de pyAudioAnalysis

Analyzing file 1 of 8: data\0.mp3
Analyzing file 2 of 8: data\1.mp3
Analyzing file 3 of 8: data\2.mp3
Analyzing file 4 of 8: data\3.mp3
Analyzing file 5 of 8: data\4.mp3
Analyzing file 6 of 8: data\5.mp3
Analyzing file 7 of 8: data\6.mp3
Analyzing file 8 of 8: data\7.mp3
Feature extraction complexity ratio: 45.3 x realtime


Normalisation:

In [3]:
songs = pd.DataFrame(songs[0], columns=songs[2])
songs = (songs - songs.mean()) / songs.std()

In [4]:
all_features = list(songs.columns)

print(f"Nombre d'attributs: {len(all_features)}")
print(all_features)

Nombre d'attributs: 138
['zcr_mean', 'energy_mean', 'energy_entropy_mean', 'spectral_centroid_mean', 'spectral_spread_mean', 'spectral_entropy_mean', 'spectral_flux_mean', 'spectral_rolloff_mean', 'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_mean', 'mfcc_5_mean', 'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean', 'mfcc_10_mean', 'mfcc_11_mean', 'mfcc_12_mean', 'mfcc_13_mean', 'chroma_1_mean', 'chroma_2_mean', 'chroma_3_mean', 'chroma_4_mean', 'chroma_5_mean', 'chroma_6_mean', 'chroma_7_mean', 'chroma_8_mean', 'chroma_9_mean', 'chroma_10_mean', 'chroma_11_mean', 'chroma_12_mean', 'chroma_std_mean', 'delta zcr_mean', 'delta energy_mean', 'delta energy_entropy_mean', 'delta spectral_centroid_mean', 'delta spectral_spread_mean', 'delta spectral_entropy_mean', 'delta spectral_flux_mean', 'delta spectral_rolloff_mean', 'delta mfcc_1_mean', 'delta mfcc_2_mean', 'delta mfcc_3_mean', 'delta mfcc_4_mean', 'delta mfcc_5_mean', 'delta mfcc_6_mean', 'delta mfcc_7_mean', 'delta mfcc

## 2. Sélection des attributs

### Fonctions

In [5]:
def l2_distance(a, b):
    """Calcule la distance L2 (euclidienne) entre deux vecteurs"""
    return np.linalg.norm(a - b)

In [6]:
def ideal_rank(n):
    """Retourne un classement idéal, c'est à dire un suite de 1 à n - 1"""
    return list(range(1, n))

In [7]:
def rank(df, features):
    """Retourne un classement de distances des vecteurs d'attributs"""
    values = df[features].values
    distances = ideal_rank(len(values))
    distances.sort(key=lambda i: l2_distance(values[0], values[i]))
    return distances

In [8]:
def compare_rank(a, b):
    """Retourne les informations du calcul d'un coefficient de Spearman pour deux classements"""
    return spearmanr(a, b)

In [9]:
def rank_score(df, features):
    """Retourne les informations du calcul d'un coefficient de Spearman pour un classement idéal et un classement de test"""
    return compare_rank(ideal_rank(len(df)), rank(df, features))

In [10]:
def show_rank_results(df, features, name=""):
    """Affiche les informations d'un classement"""
    print(f"{name}:")
    print(f"\tClassement obtenu: {rank(df, features)}")
    print(f"\tCorrélation obtenue: {rank_score(df, features)}")

### Première recherche arbitraire

Définition de quelques groupes d'attributs potentiellement intéressants:

In [11]:
mean_features = [feature for feature in all_features if feature.endswith("mean")]
mfcc_features = [feature for feature in mean_features if feature.startswith("mfcc")]
bpm_features = ["bpm"]

show_rank_results(songs, all_features, "all")
show_rank_results(songs, mean_features, "mean")
show_rank_results(songs, mfcc_features, "mfcc")
show_rank_results(songs, bpm_features, "bpm")

all:
	Classement obtenu: [1, 4, 5, 2, 6, 3, 7]
	Corrélation obtenue: SignificanceResult(statistic=0.6071428571428572, pvalue=0.1482311614811614)
mean:
	Classement obtenu: [1, 4, 5, 6, 2, 3, 7]
	Corrélation obtenue: SignificanceResult(statistic=0.4642857142857144, pvalue=0.2939341076002517)
mfcc:
	Classement obtenu: [1, 2, 6, 4, 3, 5, 7]
	Corrélation obtenue: SignificanceResult(statistic=0.7500000000000002, pvalue=0.05218140045705776)
bpm:
	Classement obtenu: [5, 1, 2, 3, 6, 4, 7]
	Corrélation obtenue: SignificanceResult(statistic=0.5714285714285715, pvalue=0.1802019889115274)


On constate que l'utilisation des moyennes de MFCC fournit le meilleur résultat, avec un coefficient de ~**0.75**.

### Recherche itérative

In [12]:
def iterative_search(df, features, n, verbose=False):
    """Effectue une recherche itérative avec n tours, en ajoutant le meilleur attribut à chaque tour"""
    test_features = features.copy()
    selected_features = list()

    best_score = 0
    for i in range(1, n + 1):
        scores = [rank_score(df, selected_features + [f]).statistic for f in test_features]

        best_index = np.argmax(scores)
        best_score = scores[best_index]
        best_feature = test_features.pop(best_index)
        
        selected_features.append(best_feature)

        if verbose:
            print(f"Step [{i}/{n}]")
            show_rank_results(songs, selected_features, name=str(selected_features))
            
    return (best_score, selected_features)

In [13]:
iterative_search(songs, all_features, 15, verbose=True)

Step [1/15]
['spectral_centroid_mean']:
	Classement obtenu: [1, 2, 3, 4, 5, 6, 7]
	Corrélation obtenue: SignificanceResult(statistic=1.0, pvalue=0.0)
Step [2/15]
['spectral_centroid_mean', 'delta spectral_flux_mean']:
	Classement obtenu: [1, 2, 3, 4, 5, 6, 7]
	Corrélation obtenue: SignificanceResult(statistic=1.0, pvalue=0.0)
Step [3/15]
['spectral_centroid_mean', 'delta spectral_flux_mean', 'zcr_mean']:
	Classement obtenu: [1, 2, 3, 4, 6, 5, 7]
	Corrélation obtenue: SignificanceResult(statistic=0.9642857142857145, pvalue=0.0004541491691941689)
Step [4/15]
['spectral_centroid_mean', 'delta spectral_flux_mean', 'zcr_mean', 'spectral_spread_mean']:
	Classement obtenu: [1, 2, 3, 4, 5, 6, 7]
	Corrélation obtenue: SignificanceResult(statistic=1.0, pvalue=0.0)
Step [5/15]
['spectral_centroid_mean', 'delta spectral_flux_mean', 'zcr_mean', 'spectral_spread_mean', 'mfcc_3_mean']:
	Classement obtenu: [1, 2, 3, 4, 5, 6, 7]
	Corrélation obtenue: SignificanceResult(statistic=1.0, pvalue=0.0)
Step [

(1.0,
 ['spectral_centroid_mean',
  'delta spectral_flux_mean',
  'zcr_mean',
  'spectral_spread_mean',
  'mfcc_3_mean',
  'energy_mean',
  'energy_entropy_mean',
  'spectral_entropy_mean',
  'mfcc_4_mean',
  'mfcc_1_mean',
  'spectral_flux_mean',
  'mfcc_6_mean',
  'mfcc_11_mean',
  'spectral_rolloff_mean',
  'chroma_2_mean'])

## Conclusion

On constate que l'on peut obtenir un classement parfait dès l'utilisation d'un seul attribut, la `spectral_centroid_mean`. Au delà, quasiment toutes les meilleures combinaisons itératives jusqu'à 15 permettent d'obtenir un score parfait. Globalement, les résultats de cette sélection d'attributs sont très satisfaisants.

Ce laboratoire nous a montré qu'il est facile d'apréhender des données audio à l'aide de la librairie `pyAudioAnalysis` et d'obtenir un système de recommendation rudimentaire mais tout de même satisfaisant; cela a probablement été facilité par la taille modeste de l'échantillon.